# Limpeza dos Dados do CAGR

## Importaçao dos Dados

In [1]:
import pandas as pd

df = pd.read_csv('Dados CAGR.csv',sep='\t',index_col=0)

df.head()

,Ementa,Currículo,Disciplina,Turma,Nome da Disciplina,Horas Aula,Vagas Ofertadas,Vagas Ocupadas,Alunos Especiais,Saldo Vagas,Pedidos sem vaga,Horários,Professor
1,NaN,NaN,ACL5130,01102A,Biossegurança e Boas Práticas de Laboratório ...,36.0,14.0,14.0,0.0,LOTADA,NaN,3.1620-2 / CCS-CCS925,Flavia Martinello
2,NaN,NaN,ACL5130,01102B,Biossegurança e Boas Práticas de Laboratório ...,36.0,14.0,14.0,0.0,LOTADA,NaN,3.0820-2 / CCS-CCS919,Flavia Martinello
3,NaN,NaN,ACL5130,01102C,Biossegurança e Boas Práticas de Laboratório ...,36.0,14.0,14.0,0.0,LOTADA,NaN,5.1620-2 / CCS-CCS927,Flavia Martinello
4,NaN,NaN,ACL5130,01102D,Biossegurança e Boas Práticas de Laboratório ...,36.0,14.0,14.0,0.0,LOTADA,NaN,6.1620-2 / CCS-CCS927,Daniela Cristina Tartari
5,NaN,NaN,ACL5130,01102E,Biossegurança e Boas Práticas de Laboratório ...,36.0,14.0,14.0,0.0,LOTADA,NaN,3.1330-2 / CCS-CCS921,Flavia Martinello


In [2]:
# As 20 matérias com mais pedidos sem vaga

df.groupby('Disciplina').sum().sort_values("Pedidos  sem vaga", ascending=False).head(20)

,Ementa,Currículo,Horas Aula,Vagas Ofertadas,Vagas Ocupadas,Alunos Especiais,Pedidos sem vaga
Disciplina,,,,,,,
MTM3100,0.0,0.0,2088.0,1624.0,1602.0,0.0,366.0
EFC5556,0.0,0.0,288.0,80.0,80.0,0.0,136.0
LSB7904,0.0,0.0,1512.0,392.0,306.0,0.0,130.0
MTM3111,0.0,0.0,648.0,511.0,505.0,0.0,129.0
EFC5804,0.0,0.0,72.0,30.0,30.0,0.0,108.0
CNM6001,0.0,0.0,144.0,91.0,90.0,0.0,96.0
EFC5801,0.0,0.0,504.0,141.0,141.0,0.0,86.0
MTM5512,0.0,0.0,936.0,708.0,703.0,0.0,67.0
EFC5806,0.0,0.0,72.0,30.0,30.0,0.0,66.0


In [3]:
def get_curso(m):
    return m.split('*')[-1]
    

df["Curso"] = df["Nome da Disciplina"].apply(get_curso)

df["Curso"]

1       Biossegurança e Boas Práticas de Laboratório  ...
2       Biossegurança e Boas Práticas de Laboratório  ...
3       Biossegurança e Boas Práticas de Laboratório  ...
4       Biossegurança e Boas Práticas de Laboratório  ...
5       Biossegurança e Boas Práticas de Laboratório  ...
6                                   Parasitologia Clínica
7                                   Parasitologia Clínica
8                                   Parasitologia Clínica
9                                   Parasitologia Clínica
10                                  Parasitologia Clínica
11                                  Parasitologia Clínica
12                                      Citologia Clínica
13                                      Citologia Clínica
14                                      Citologia Clínica
15                                      Citologia Clínica
16                                      Citologia Clínica
17                                     Bioquímica Clínica
18            

In [4]:
def get_centro(d):
    return d[0:3]

df["Centro"] = df["Disciplina"].apply(get_centro)

df.groupby('Centro').sum().sort_values("Pedidos  sem vaga", ascending=False).describe()

,Ementa,Currículo,Horas Aula,Vagas Ofertadas,Vagas Ocupadas,Alunos Especiais,Pedidos sem vaga
count,70.0,70.0,70.000000,70.000000,70.000000,70.000000,70.000000
mean,0.0,0.0,6661.014286,2068.871429,1534.728571,0.114286,73.285714
std,0.0,0.0,11493.917673,1770.206161,1390.279077,0.956183,122.509762
min,0.0,0.0,0.000000,30.000000,1.000000,0.000000,0.000000
25%,0.0,0.0,2304.000000,806.750000,640.750000,0.000000,9.250000
50%,0.0,0.0,4509.000000,1490.500000,1098.000000,0.000000,39.500000
75%,0.0,0.0,8015.250000,2960.250000,2064.500000,0.000000,78.250000
max,0.0,0.0,94920.000000,7655.000000,6564.000000,8.000000,730.000000


In [5]:
import seaborn as sns

ranking_centros = df.groupby('Centro').mean().sort_values('Vagas  Ofertadas', ascending = False).index

sns.barplot(x = 'Centro', y = 'Vagas  Ofertadas', data= df, order = ranking_centros)

/home/lucas/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [61]:
def get_horarios(h):
    if isinstance(h, str):
        return h.replace(' / ','|').split(' ')
    else:
        return []

df["Limpeza Horário"] = df['Horários'].apply(get_horarios)

df["Limpeza Horário"]

def get_weekday(h):
    if len(h) >= 1:
        return [x[0] for x in h if len(x) > 0]
    else:
        return None

def get_time(h):
    if len(h) >= 1:
        return [x[2:6] for x in h if len(x) > 0]
    else:
        return None
    
def get_creditos(h):
    if len(h) >= 1:
        return [x[7] for x in h if len(x) > 7]
    else:
        return None
    
def get_local(h):
    if len(h) >= 1:
        return [x.split('|')[-1] for x in h if len(x) > 0]
    else:
        return None
    
    

df["Dia(s) da Semana"] = df["Limpeza Horário"].apply(get_weekday)

df["Horário no dia"] = df["Limpeza Horário"].apply(get_time)

df["Créditos no horário"] = df["Limpeza Horário"].apply(get_creditos)

df["Sala"] = df["Limpeza Horário"].apply(get_local)

df.drop(["Limpeza Horário"], axis='columns')

df.head(10)

,Ementa,Currículo,Disciplina,Turma,Nome da Disciplina,Horas Aula,Vagas Ofertadas,Vagas Ocupadas,Alunos Especiais,Saldo Vagas,Pedidos sem vaga,Horários,Professor,Curso,Centro,Limpeza Horário,Dia(s) da Semana,Horário no dia,Créditos no horário,Sala
1,NaN,NaN,ACL5130,01102A,Biossegurança e Boas Práticas de Laboratório ...,36.0,14.0,14.0,0.0,LOTADA,NaN,3.1620-2 / CCS-CCS925,Flavia Martinello,Biossegurança e Boas Práticas de Laboratório ...,ACL,[3.1620-2|CCS-CCS925],[3],[1620],[2],[CCS-CCS925]
2,NaN,NaN,ACL5130,01102B,Biossegurança e Boas Práticas de Laboratório ...,36.0,14.0,14.0,0.0,LOTADA,NaN,3.0820-2 / CCS-CCS919,Flavia Martinello,Biossegurança e Boas Práticas de Laboratório ...,ACL,[3.0820-2|CCS-CCS919],[3],[0820],[2],[CCS-CCS919]
3,NaN,NaN,ACL5130,01102C,Biossegurança e Boas Práticas de Laboratório ...,36.0,14.0,14.0,0.0,LOTADA,NaN,5.1620-2 / CCS-CCS927,Flavia Martinello,Biossegurança e Boas Práticas de Laboratório ...,ACL,[5.1620-2|CCS-CCS927],[5],[1620],[2],[CCS-CCS927]
4,NaN,NaN,ACL5130,01102D,Biossegurança e Boas Práticas de Laboratório ...,36.0,14.0,14.0,0.0,LOTADA,NaN,6.1620-2 / CCS-CCS927,Daniela Cristina Tartari,Biossegurança e Boas Práticas de Laboratório ...,ACL,[6.1620-2|CCS-CCS927],[6],[1620],[2],[CCS-CCS927]
5,NaN,NaN,ACL5130,01102E,Biossegurança e Boas Práticas de Laboratório ...,36.0,14.0,14.0,0.0,LOTADA,NaN,3.1330-2 / CCS-CCS921,Flavia Martinello,Biossegurança e Boas Práticas de Laboratório ...,ACL,[3.1330-2|CCS-CCS921],[3],[1330],[2],[CCS-CCS921]
6,NaN,NaN,ACL5131,05102A,Parasitologia Clínica,72.0,9.0,9.0,0.0,LOTADA,NaN,2.1330-4 / CCS-ACL005,Karin Silva Caumo,Parasitologia Clínica,ACL,[2.1330-4|CCS-ACL005],[2],[1330],[4],[CCS-ACL005]
7,NaN,NaN,ACL5131,05102B,Parasitologia Clínica,72.0,9.0,8.0,0.0,1,NaN,2.1330-1 / CCS-ACL005 3.0820-3 / CCS-ACL011,Karin Silva Caumo,Parasitologia Clínica,ACL,"[2.1330-1|CCS-ACL005, , 3.0820-3|CCS-ACL011]","[2, 3]","[1330, 0820]","[1, 3]","[CCS-ACL005, CCS-ACL011]"
8,NaN,NaN,ACL5131,05102C,Parasitologia Clínica,72.0,8.0,8.0,0.0,LOTADA,NaN,2.1330-1 / CCS-ACL005 4.1330-3 / CCS-ACL011,Lenilza Mattos Lima,Parasitologia Clínica,ACL,"[2.1330-1|CCS-ACL005, , 4.1330-3|CCS-ACL011]","[2, 4]","[1330, 1330]","[1, 3]","[CCS-ACL005, CCS-ACL011]"
9,NaN,NaN,ACL5131,05102D,Parasitologia Clínica,72.0,9.0,9.0,0.0,LOTADA,1.0,2.1330-1 / CCS-ACL005 3.1330-3 / CCS-ACL011,Lenilza Mattos Lima,Parasitologia Clínica,ACL,"[2.1330-1|CCS-ACL005, , 3.1330-3|CCS-ACL011]","[2, 3]","[1330, 1330]","[1, 3]","[CCS-ACL005, CCS-ACL011]"
10,NaN,NaN,ACL5131,05102E,Parasitologia Clínica,72.0,8.0,8.0,0.0,LOTADA,NaN,2.1330-1 / CCS-ACL005 4.0820-3 / CCS-ACL011,Lenilza Mattos Lima,Parasitologia Clínica,ACL,"[2.1330-1|CCS-ACL005, , 4.0820-3|CCS-ACL011]","[2, 4]","[1330, 0820]","[1, 3]","[CCS-ACL005, CCS-ACL011]"


In [62]:
df.to_csv("Dados com Horários - CAGR.csv", sep=";")